# Baseline: NIRISS Imaging Tutorial Notebook

## Table of Contents
1. [Introduction](#intro)
2. [Examining uncalibrated data products](#examine)
3. [Stage 1 Processing](#det1)
4. [Stage 2 Processing](#image2)
5. [Stage 3 Processing](#image3)
6. [Visualize Detected Sources](#detections)

## <a id='intro'>1. Introduction</a>
In this notebook, we will process a [NIRISS imaging dataset](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-observing-modes/niriss-imaging) through the [JWST calibration pipeline](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview). The example dataset is from [Program ID](https://www.stsci.edu/jwst/science-execution/program-information) 1475 (PI: Boyer, CoI: Volk) which is a sky flat calibration program. NIRCam is used as the primary instrument with NIRISS as a [coordinated parallel instrument](https://jwst-docs.stsci.edu/methods-and-roadmaps/jwst-parallel-observations/jwst-coordinated-parallels-roadmap). The NIRISS imaging dataset uses a 17-step dither pattern.

For illustrative purposes, we focus on data taken through the NIRISS [F150W filter](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-instrumentation/niriss-filters) and start with uncalibrated data products. The files are named
`jw01475006001_02201_000nn_nis_uncal.fits`, where *nn* refers to the dither step number which ranges from 01 - 17. See the ["File Naming Schemes" documentation](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/file_naming.html) to learn more about the file naming convention.

In this notebook, we assume all uncalibrated fits files are saved in a directory named `1475_f150w`.

### Install pipeline and dependencies

Make sure the [latest version of the pipeline is installed](https://jwst-pipeline.readthedocs.io/en/stable/index.html) and that the [Calibration Reference Data System (CRDS)](https://jwst-crds.stsci.edu/static/users_guide/index.html) is configured so the most up-to-date calibration files are used to process the dataset. This notebook was created with pipeline version `1.12.3`. 

This notebook uses the [`jdaviz` package](https://jdaviz.readthedocs.io/en/latest/index.html) to visualize the datasets. To [install `jdaviz`](https://jdaviz.readthedocs.io/en/latest/installation.html) in your current environment:
```
pip install jdaviz --upgrade
```

### Imports

In [ ]:
import os, glob
import numpy as np

# Configure CRDS
os.environ["CRDS_PATH"] = 'crds_cache'
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
import crds

# for JWST calibration pipeline
import jwst
from jwst import datamodels
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline
from jwst.associations import asn_from_list, mkpool
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
# For visualizing images
from jdaviz import Imviz

# Astropy routines for visualizing detected sources:
from astropy.table import Table
from astropy.coordinates import SkyCoord

# To confirm which version of the pipeline you're running:
print(f"jwst pipeline version: {jwst.__version__}")

### Define program parameters

These will be used for generating the assocation file for Stage 3

In [ ]:
# APT program ID number:
pid = '01475'

data_dir = '1475_f150w/'

***
## <a id='examine'>2. Examining uncalibrated data products</a>

In [ ]:
uncal_files = sorted(glob.glob(os.path.join(data_dir,'*_uncal.fits')))

### Look at the first file to determine exposure parameters and practice using JWST datamodels

In [ ]:
# print file name
print(uncal_files[0])

# Open file as JWST datamodel
examine = datamodels.open(uncal_files[0])

# Print out exposure info
print("Instrument: " + examine.meta.instrument.name)
print("Filter: " + examine.meta.instrument.filter)
print("Pupil: " + examine.meta.instrument.pupil)
print("Number of integrations: {}".format(examine.meta.exposure.nints))
print("Number of groups: {}".format(examine.meta.exposure.ngroups))
print("Readout pattern: " + examine.meta.exposure.readpatt)
print("Dither position number: {}".format(examine.meta.dither.position_number))
print("\n")

From the above, we confirm that the data file is for the NIRISS instrument using the `F150W` filter in the [Pupil Wheel](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-instrumentation/niriss-pupil-and-filter-wheels) crossed with the `CLEAR` filter in the Filter Wheel. This observation uses the [`NIS` readout pattern](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-instrumentation/niriss-detector-overview/niriss-detector-readout-patterns), 16 groups per integration, and 1 integration per exposure. This data file is the 1st dither position in this exposure sequence. For more information about how JWST exposures are defined by up-the-ramp sampling, see the [Understanding Exposure Times JDox article](https://jwst-docs.stsci.edu/understanding-exposure-times).

This metadata will be the same for all exposures in this observation other than the dither position number.

### Display uncalibrated image

We can visualize an uncalibrated dataset that will show detector artifacts that will be removed when calibrating the data through the `DETECTOR1` stage of the pipeline. Uncalibrated data files thus are 4D: nintegrations x ngroups x nrows x ncols. Here, we are visualizing the full detector (i.e., all columns and rows) and the 1st group.

We are using the [Imviz tool](https://jdaviz.readthedocs.io/en/latest/imviz/index.html) within the `jdaviz` package to visualize the NIRISS image.

In [ ]:
# Create an Imviz instance and set up default viewer
imviz_uncal = Imviz()
viewer_uncal = imviz_uncal.default_viewer

# Read in the science array for our visualization dataset:
uncal_science = examine.data

# Load the dataset into Imviz
imviz_uncal.load_data(uncal_science[0,0,:,:])

# Visualize the dataset:
imviz_uncal.show()

Adjust settings for the viewer:

In [ ]:
viewer_uncal.stretch = 'sqrt'
viewer_uncal.set_colormap('Viridis')
viewer_uncal.cuts = '99.5%'

***
## <a id='det1'>3. Stage 1 Processing </a>
Run the datasets through the [Detector1 stage of the pipeline](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_detector1) to apply detector level calibrations and create a countrate data product where slopes are fitted to the integration ramps. These `*_rate.fits` products are 2D (nrows x ncols), averaged over all integrations. 3D countrate data products (`*_rateints.fits`) are also created (nintegrations x nrows x ncols) which have the fitted ramp slopes for each integration.

The [pipeline documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/user_documentation/running_pipeline_python.html) discusses how to run the pipeline in the Python Interface, including how to configure pipeline steps and override reference files. By returning the results of the pipeline to a variable, the pipeline returns a datamodel. Note that the [`pipeline.call()` method is preferred over the `pipeline.run()` method](https://jwst-pipeline.readthedocs.io/en/latest/jwst/user_documentation/running_pipeline_python.html#advanced-use-pipeline-run-vs-pipeline-call).

By default, [all steps in the Detector1 stage of the pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html) are run for NIRISS except: the `ipc` correction step and the `gain_scale` step. Note that while the [`persistence` step](https://jwst-pipeline.readthedocs.io/en/latest/jwst/persistence/description.html) is set to run by default, this step does not automatically correct the science data for persistence. The `persistence` step creates a `*_trapsfilled.fits` file which is a model that records the number of traps filled at each pixel at the end of an exposure. This file would be used as an input to the `persistence` step, via the `input_trapsfilled` argument, to correct a science exposure for persistence. Since persistence is not well calibrated for NIRISS, we do not perform a persistence correction and thus turn off this step to speed up calibration and to not create files that will not be used in the subsequent analysis. This step can be turned off when running the pipeline in Python by doing:
```
rate_result = Detector1Pipeline.call(uncal,
                                     steps={'persistence': {'skip': True}})
```

The [charge_migration step](https://jwst-pipeline.readthedocs.io/en/latest/jwst/charge_migration/index.html#charge-migration-step) is particularly important for NIRISS images to mitigate apparent flux loss in resampled images due to the spilling of charge from a central pixel into its neighboring pixels (see [Goudfrooij et al. 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv231116301G/abstract) for details). Charge migration occurs when the accumulated charge in a central pixel exceeds a certain signal limit, which is ~25,000 ADU. This step is turned on by default for NIRISS imaging, Wide Field Slitless Spectroscopy [(WFSS)](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-observing-modes/niriss-wide-field-slitless-spectroscopy), and Aperture Masking Interferometry [(AMI)](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-observing-modes/niriss-aperture-masking-interferometry) modes when using CRDS contexts of `jwst_1159.pmap` or later. Different signal limits for each filter are provided by the [pars-chargemigrationstep parameter files](https://jwst-crds.stsci.edu). Users can specify a different signal limit by running this step with the `signal_threshold` flag and entering another signal limit in units of ADU.

As of CRDS context `jwst_1155.pmap` and later, the [`jump` step](https://jwst-pipeline.readthedocs.io/en/latest/api/jwst.jump.JumpStep.html) of the `DETECTOR1` stage of the pipeline will remove residuals associated with [snowballs](https://jwst-docs.stsci.edu/data-artifacts-and-features/snowballs-and-shower-artifacts) for NIRISS imaging, WFSS, and AMI modes. The default parameters for this correction, where `expand_large_events` set to `True` turns on the snowball halo removal algorithm, are specified in the `pars-jumpstep` parameter reference files. Users may wish to alter parameters to optimize removal of snowball residuals. Available parameters are discussed in the [Detection and Flagging of Showers and Snowballs in JWST Technical Report (Regan 2023)](https://www.stsci.edu/files/live/sites/www/files/home/jwst/documentation/technical-documents/_documents/JWST-STScI-008545.pdf).

In [ ]:
# Define directory to save output from detector1
det1_dir = 'detector1/'
# Create directory if it does not exist
if not os.path.isdir(det1_dir):
    os.mkdir(det1_dir)

# Run Detector1 stage of pipeline, specifying:
# output directory to save *_rate.fits files
# save_results flag set to True so the rate files are saved
# skipping the persistence step
    
for uncal in uncal_files:
    rate_result = Detector1Pipeline.call(uncal,
                                         output_dir=det1_dir,
                                         save_results=True,
                                         steps={'persistence': {'skip': True}})

### Identify `*_rate.fits` files

In [ ]:
rate_files = sorted(glob.glob(os.path.join(det1_dir,'*_rate.fits')))

### Verify which pipeline steps were run and which calibration reference files were applied

The header contains information about which calibration steps were completed and skipped and which reference files were used to process the data.

In [ ]:
# Read in file as datamodel
rate_f = datamodels.open(rate_files[0])

rate_f.meta.cal_step.instance

Check which reference files were used to calibrate the dataset:

In [ ]:
rate_f.meta.ref_file.instance

### Display rate image

Visualize a countrate image, using the dataset from the first dither position as an example.

In [ ]:
# Create an Imviz instance and set up default viewer
imviz_rate = Imviz()
viewer_rate = imviz_rate.default_viewer

# Read in the science array for our visualization dataset:
rate_science = rate_f.data

# Load the dataset into Imviz
imviz_rate.load_data(rate_science)

# Visualize the dataset:
imviz_rate.show()

In [ ]:
viewer_rate.stretch = 'sqrt'
viewer_rate.set_colormap('Viridis')
viewer_rate.cuts = '95%'

***
## <a id='image2'>3. Stage 2 Processing </a>

In the [Image2 stage of the pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html), calibrated unrectified data products are created (`*_cal.fits` or `*_calints.fits` files, depending on whether the input files are `*_rate.fits` or `*_rateints.fits`). 

In this pipeline processing stage, the [world coordinate system (WCS)](https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/index.html#assign-wcs-step) is assigned, the data are [flat fielded](https://jwst-pipeline.readthedocs.io/en/latest/jwst/flatfield/index.html#flatfield-step), and a [photometric calibration](https://jwst-pipeline.readthedocs.io/en/latest/jwst/photom/index.html#photom-step) is applied to convert from units of countrate (ADU/s) to surface brightness (MJy/sr). 

By default, the [background subtraction step](https://jwst-pipeline.readthedocs.io/en/latest/jwst/background_step/index.html#background-step) and the [resampling step](https://jwst-pipeline.readthedocs.io/en/latest/jwst/resample/index.html#resample-step) are turned off for NIRISS at this stage of the pipeline.

In [ ]:
# Define directory to save output from Image2
image2_dir = 'image2/'
# Create directory if it does not exist
if not os.path.isdir(image2_dir):
    os.mkdir(image2_dir)

# Run Image2 stage of pipeline, specifying:
# output directory to save *_cal.fits files
# save_results flag set to True so the rate files are saved

for rate in rate_files:
    cal_result = Image2Pipeline.call(rate,
                                     output_dir=image2_dir,
                                     save_results=True)

### Identify `*_cal.fits` files

In [ ]:
cal_files = sorted(glob.glob(os.path.join(image2_dir,'*_cal.fits')))

### Verify which pipeline steps were run

In [ ]:
cal_f = datamodels.open(cal_files[0])

cal_f.meta.cal_step.instance

Check which reference files were used to calibrate the dataset:

In [ ]:
cal_f.meta.ref_file.instance

### Display cal image

Visualize a calibrated image, using the dataset from the first dither position as an example.

In [ ]:
# Create an Imviz instance and set up default viewer
imviz_cal = Imviz()
viewer_cal = imviz_cal.default_viewer

# Read in the science array for our visualization dataset:
cal_science = cal_f.data

# Load the dataset into Imviz
imviz_cal.load_data(cal_science)

# Visualize the dataset:
imviz_cal.show()

In [ ]:
viewer_cal.stretch = 'sqrt'
viewer_cal.set_colormap('Viridis')
viewer_cal.cuts = '95%'

***
## <a id='image3'>3. Stage 3 Processing </a>

In the [Image3 stage of the pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html), the individual `*_cal.fits` files for each of the dither positions are combined to one single distortion corrected image. First, an [Association](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/overview.html) needs to be created to inform the pipeline that these individual exposures are linked together. 

By default, the `IMAGE3` stage of the pipeline performs the following steps on NIRISS data: 
- [tweakreg](https://jwst-pipeline.readthedocs.io/en/latest/jwst/tweakreg/index.html#tweakreg-step) - creates source catalogs of pointlike sources for each input image. The source catalog for each input image is compared to each other to derive coordinate transforms to align the images relative to each other.
   - As of CRDS context `jwst_1156.pmap` and later, the `pars-tweakreg` parameter reference file for NIRISS performs an absolute astrometric correction to GAIA data release 3 by default (i.e., the `abs_refcat` parameter is set to `GAIADR3`). Though this default correction generally improves results compared with not doing this alignment, it can sometimes result in poor performance in crowded or sparse fields, so users are encouraged to check astrometric accuracy and revisit this step if necessary.
   - As of pipeline version 1.12.5, the default source finding algorithm is `DAOStarFind` which can result in up to 0.5 pix uncertainties in the centroids for undersampled PSFs, like the NIRISS PSFs at short wavelengths [(Goudfrooij 2022)](https://www.stsci.edu/files/live/sites/www/files/home/jwst/documentation/technical-documents/_documents/JWST-STScI-008116.pdf). There are plans to update the default algorithm to `IRAFStarFind` in future pipeline versions.
- [skymatch](https://jwst-pipeline.readthedocs.io/en/latest/jwst/skymatch/index.html#skymatch-step) - measures the background level from the sky to use as input into the subsequent `outlier detection` and `resample` steps.
- [outlier detection](https://jwst-pipeline.readthedocs.io/en/latest/jwst/outlier_detection/index.html#outlier-detection-step) - flags any remaining cosmic rays, bad pixels, or other artifacts not already flagged during the `DETECTOR1` stage of the pipeline, using all input images to create a median image so that outliers in individual images can be identified.
- [resample](https://jwst-pipeline.readthedocs.io/en/latest/jwst/resample/index.html#resample-step) - resamples each input image based on its WCS and distortion information and creates a single undistorted image.
- [source catalog](https://jwst-pipeline.readthedocs.io/en/latest/jwst/source_catalog/index.html#source-catalog-step) - creates a catalog of detected sources along with measured photometries and morphologies (i.e., point-like vs extended). Useful for quicklooks, but optimization is likely needed for specific science cases.


### Create Association File

An association file lists the exposures to calibrated together in `Stage 3` of the pipeline. Note that an association file is available for download from MAST, with a filename of `*_asn.json`. Here we show how to create an association file to point to the data products created when processing data through the pipeline. Note that the output products will have a rootname that is specified by the `product_name` in the association file. For this tutorial, the rootname of the output products will be `image3_association`.

In [ ]:
# Create a Level 3 Association
associations = asn_from_list.asn_from_list(cal_files, 
                                           rule=DMS_Level3_Base, 
                                           product_name='image3_association')

associations.data['asn_type'] = 'image3'
associations.data['program'] = pid

# Format association as .json file
asn_filename, serialized = associations.dump(format="json")

# Write out association file
with open(asn_filename,"w") as fd:
    fd.write(serialized)

### Run Image3 stage of the pipeline

Given the grouped exposures in the association file, the `Image3` stage of the pipeline will produce:
- a `*_cr.fits` file produced by the `outlier_detection` step, where the `DQ` array marks the pixels flagged as outliers.
- a final combined, rectified image with name `*_i2d.fits`,
- a source catalog with name `*_cat.ecsv`,
- a segmentation map file (`*_segm.fits`) which has integer values at the pixel locations where a source is detected where the pixel values match the source ID number in the catalog.

In [ ]:
# Define directory to save output from Image2
image3_dir = 'image3/'
# Create directory if it does not exist
if not os.path.isdir(image3_dir):
    os.mkdir(image3_dir)

# Run Stage 3
i2d_result = Image3Pipeline.call(asn_filename, 
                                 output_dir=image3_dir,
                                 save_results=True)

### Verify which pipeline steps were run

In [ ]:
# Identify *_i2d file and open as datamodel
i2d = glob.glob(os.path.join(image3_dir,"*_i2d.fits"))[0]
i2d_f = datamodels.open(i2d)

i2d_f.meta.cal_step.instance

Check which reference files were used to calibrate the dataset:

In [ ]:
i2d_f.meta.ref_file.instance

### Display combined image

Visualize the drizzle-combined image.

In [ ]:
# Create an Imviz instance and set up default viewer
imviz_i2d = Imviz()
viewer_i2d = imviz_i2d.default_viewer

# Read in the science array for our visualization dataset:
i2d_science = i2d_f.data

# Load the dataset into Imviz
imviz_i2d.load_data(i2d_science)

# Visualize the dataset:
imviz_i2d.show()

In [ ]:
viewer_i2d.stretch = 'sqrt'
viewer_i2d.set_colormap('Viridis')
viewer_i2d.cuts = '95%'

## <a id='detections'>Visualize Detected Sources</a>
Using the source catalog created by the `IMAGE3` stage of the pipeline, mark the detected sources, using different markers for point sources and extended sources. The source catalog is saved in `image3/image3_association_cat.ecsv` file. We will need to read in the `i2d` file again to make sure the world coordinate system (WCS) info is read in.

### Read in catalog file and identify point/extended sources

In [ ]:
catalog_file = glob.glob(os.path.join(image3_dir,"*_cat.ecsv"))[0]
catalog = Table.read(catalog_file)

# To identify point/extended sources, use the 'is_extended' column in the source catalog
pt_src, = np.where(catalog['is_extended'] == False)
ext_src, = np.where(catalog['is_extended'] == True)

# Define coordinates of point and extended sources
pt_coord = Table({'coord':[SkyCoord(ra=catalog['sky_centroid'][pt_src].ra,
                                    dec=catalog['sky_centroid'][pt_src].dec)]})
ext_coord = Table({'coord':[SkyCoord(ra=catalog['sky_centroid'][ext_src].ra,
                                    dec=catalog['sky_centroid'][ext_src].dec)]})

### Mark the extended and point sources on the image

Display combined image:

In [ ]:
# Read in i2d file to Imviz
imviz_cat = Imviz()
viewer_cat = imviz_cat.default_viewer
imviz_cat.load_data(i2d)

# Adjust settings for viewer
viewer_cat.stretch = 'sqrt'
viewer_cat.set_colormap('Viridis')
viewer_cat.cuts = '95%'


imviz_cat.show()

Point sources will be marked by small pink circles and extended sources will be marked by larger white circles.

In [ ]:
# Add marker for point sources:
viewer_cat.marker = {'color': 'pink', 'markersize':50, 'fill':False}

viewer_cat.add_markers(pt_coord, use_skycoord=True, marker_name='point_sources')

# Add marker for extended sources:
viewer_cat.marker = {'color': 'white', 'markersize':100, 'fill': False}

viewer_cat.add_markers(ext_coord, use_skycoord=True, marker_name='extended_sources')


From zoooming in on different portions of the image, we can see that some saturated point sources are erroneously labeled as point sources, some extended and saturated objects are not detected, parts of diffraction spikes are sometimes detected as "extended sources", and in some cases, the detected source centroid is offset from the center of a source. It is recommended for users to optimize source detection for their science goals using either their own tools or by updating parameters to the `source_catalog` step of the pipeline.